In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

## reading the dataset in

In [2]:
dataset_pth = "../datasets/nearest-earth-objects.csv"
df = pd.read_csv(dataset_pth) # dataframe

df.head()

,neo_id,name,absolute_magnitude,estimated_diameter_min,estimated_diameter_max,orbiting_body,relative_velocity,miss_distance,is_hazardous
0,2162117,162117 (1998 SD15),19.14,0.394962,0.883161,Earth,71745.401048,5.814362e+07,False
1,2349507,349507 (2008 QY),18.50,0.530341,1.185878,Earth,109949.757148,5.580105e+07,True
2,2455415,455415 (2003 GA),21.45,0.136319,0.304818,Earth,24865.506798,6.720689e+07,False
3,3132126,(2002 PB),20.63,0.198863,0.444672,Earth,78890.076805,3.039644e+07,False
4,3557844,(2011 DW),22.70,0.076658,0.171412,Earth,56036.519484,6.311863e+07,False


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338199 entries, 0 to 338198
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   neo_id                  338199 non-null  int64  
 1   name                    338199 non-null  object 
 2   absolute_magnitude      338171 non-null  float64
 3   estimated_diameter_min  338171 non-null  float64
 4   estimated_diameter_max  338171 non-null  float64
 5   orbiting_body           338199 non-null  object 
 6   relative_velocity       338199 non-null  float64
 7   miss_distance           338199 non-null  float64
 8   is_hazardous            338199 non-null  bool   
dtypes: bool(1), float64(5), int64(1), object(2)
memory usage: 21.0+ MB


## checking all null entries

In [4]:
df.isnull().sum()

neo_id                     0
name                       0
absolute_magnitude        28
estimated_diameter_min    28
estimated_diameter_max    28
orbiting_body              0
relative_velocity          0
miss_distance              0
is_hazardous               0
dtype: int64

## creating a subset of the original data (don't need to use all of it) -  if no nulls then we don't need to dropna()

In [5]:
df_subset = df.head(300) # selecting a subset of the dataset

df_subset.isnull().sum()

neo_id                    0
name                      0
absolute_magnitude        0
estimated_diameter_min    0
estimated_diameter_max    0
orbiting_body             0
relative_velocity         0
miss_distance             0
is_hazardous              0
dtype: int64

## we can count the number of hazardous asteroid objects and non-hazardous asteroid objects

In [6]:
df_subset["is_hazardous"].value_counts()

is_hazardous
False    255
True      45
Name: count, dtype: int64

In [7]:
y = df_subset.iloc[:, -1] # set the target set to be only the last column

print(y)

0      False
1       True
2      False
3      False
4      False
       ...  
295     True
296     True
297    False
298    False
299    False
Name: is_hazardous, Length: 300, dtype: bool


In [8]:
df_subset["is_hazardous"].value_counts()

is_hazardous
False    255
True      45
Name: count, dtype: int64

## converting target set Y to 0 or 1 for binary classification

In [9]:
df_subset["is_hazardous"] = df["is_hazardous"].astype(int)

print(df_subset["is_hazardous"])

0      0
1      1
2      0
3      0
4      0
      ..
295    1
296    1
297    0
298    0
299    0
Name: is_hazardous, Length: 300, dtype: int64


/var/folders/mw/8tv2pqjn0kx4nlmm893wbfg00000gp/T/ipykernel_4254/1818337623.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset["is_hazardous"] = df["is_hazardous"].astype(int)


## creating our feature set and target set

In [10]:
X = df_subset[["absolute_magnitude", "estimated_diameter_max"]]
y = df_subset["is_hazardous"]


print(X)

     absolute_magnitude  estimated_diameter_max
0                 19.14                0.883161
1                 18.50                1.185878
2                 21.45                0.304818
3                 20.63                0.444672
4                 22.70                0.171412
..                  ...                     ...
295               21.10                0.358129
296               21.94                0.243243
297               23.77                0.104722
298               26.78                0.026184
299               24.40                0.078350

[300 rows x 2 columns]


## we define the activation function - we are using sigmoid

In [11]:
def sigmoid(z): # g = f(x) =  1 / 1 + e^(-z)
    return 1 / (1 + np.exp(-z))

## testing activation to see if initial computation is correct

In [12]:
sigmoid(2)

0.8807970779778823

## Now we define our dense function which computes the activations of the dense layer for forward propagation where dense refers to a NN in which each layer is deeply connected to the prev. layer

In [13]:
# needs to return activation_output which is to be fed to next layer
def dense(activation_input, W, b): # activation_in is X
    units = W.shape[1]
    activation_output = np.zeros(units)
    for j in range(units):
        w = W[:, j] # same as W[j] but for clarity I will keep it as it is
        z = np.dot(w, activation_input) + b[j] # Wx + b
        activation_output = sigmoid(z)
    return activation_output

In [14]:
def relu_activation(z):
    return np.maximum(0, z)

## Now let's build a very simple 3-layer neural network using dense()

In [15]:
def custom_sequential_model(x, W1, b1, W2, b2, W3, b3, W4, b4):
    activation_layer1 = dense(x, W1, b1)
    activation_layer2 = relu_activation(dense(activation_layer1, W2, b2))
    activation_layer3 = relu_activation(dense(activation_layer2, W3, b3))
    activation_layer4 = dense(activation_layer3, W4, b4)
    return activation_layer4

## setting some random weights and biases

In [16]:
# # First hidden layer
# W1_tmp = np.array([
#     [-0.58,  0.76,  0.91],
#     [ 0.23, -0.82, -0.44]
# ])
# b1_tmp = np.array([-0.37,  0.51, -0.63])

# # Second hidden layer
# W2_tmp = np.array([
#     [ 0.71, -0.52,  0.43],
#     [-0.33,  0.61,  0.29],
#     [ 0.95, -0.79, -0.40]
# ])
# b2_tmp = np.array([0.12, -0.38,  0.76])

# # Output layer
# W3_tmp = np.array([[-0.86],
#                    [ 0.35],
#                    [-0.67]
#                    ])
# b3_tmp = np.array([0.21])

W1_tmp = np.random.uniform(low=-1.0, high=1.0, size=(2, 3))
b1_tmp = np.random.uniform(low=-1.0, high=1.0, size=(3,))

W2_tmp = np.random.uniform(low=-1.0, high=1.0, size=(3, 3))
b2_tmp = np.random.uniform(low=-1.0, high=1.0, size=(3,))

W3_tmp = np.random.uniform(low=-1.0, high=1.0, size=(3, 3))
b3_tmp = np.random.uniform(low=-1.0, high=1.0, size=(3,))


W4_tmp = np.random.uniform(low=-1.0, high=1.0, size=(3, 1))
b4_tmp = np.random.uniform(low=-1.0, high=1.0, size=(1,))



## We can define our predict function which basically makes predictions for each sample (0 or 1) -> (non-hazardous or hazardous)

In [17]:
def predict(X, W1, b1, W2, b2, W3, b3, w4, b4):
    ## m = number of training examples  - rows
    m = X.shape[0]
    predictions_buffer = np.zeros((m, 1)) # predictions buffer - we set to be 0 for each sample initially
    for i in range(m):
        predictions_buffer[i, 0] = custom_sequential_model(X[i], W1, b1, W2, b2, W3, b3, w4, b4)
    return predictions_buffer

## testing our predict function on some random samples I made up

In [18]:
X_test_data = np.array([
    [0.066765941, 0.149293183],
    [0.316632126, 0.708010957]
])
test_predictions = predict(X_test_data, W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp, W4_tmp, b4_tmp)

ValueError: setting an array element with a sequence.

In [ ]:
print(test_predictions)

In [ ]:
yhat = (test_predictions >= 0.3).astype(int)
print(f"test_predictions:\n {yhat}")

## Now for the actual predictions

In [ ]:
X = np.array(X)
X

## Forgot to normalize above, do that now to avoid skewed range of data

In [ ]:
normalized_layer = tf.keras.layers.Normalization(axis=-1)

print(normalized_layer)


In [ ]:
normalized_layer.adapt(X)

X_normalized = normalized_layer(X)

print(X_normalized)

## creating predictions using our custom model

In [ ]:
predictions = predict(X_normalized, W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp)

predictions

In [ ]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0

print(f"decisions = \n{yhat}")

In [ ]:
plt.scatter(predictions, y)
plt.show()